In [1]:
from xclib.data import data_utils
import numpy as np
import pandas as pd
from math import log2
import time

In [25]:
# Read sparse file
train_x = data_utils.read_sparse_file('ass3_parta_data/train_x.txt', force_header=True)
train_x = np.array(train_x.toarray(),dtype =int)
train_y = pd.read_csv('ass3_parta_data/train_y.txt', sep="\n", header=None).to_numpy()

test_x = data_utils.read_sparse_file('ass3_parta_data/test_x.txt', force_header=True)
test_x = np.array(test_x.toarray(),dtype =int)
test_y = pd.read_csv('ass3_parta_data/test_y.txt', sep="\n", header=None).to_numpy()

val_x = data_utils.read_sparse_file('ass3_parta_data/valid_x.txt', force_header=True)
val_x = np.array(val_x.toarray(),dtype =int)
val_y = pd.read_csv('ass3_parta_data/valid_y.txt', sep="\n", header=None).to_numpy()

/home/vivek/.local/lib/python3.6/site-packages/xclib-0.96-py3.6-linux-x86_64.egg/xclib/data/data_utils.py:173: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [3]:
def entropy(data):
    """
    Calculates the uncertanity of data :
    sparse distibuted data low entropy
    mixed distibuted data high entropy
    """
    totaldata = data.shape[0]
    
    Totalpositive = np.sum(data[:,-1]==1)
    Totalnegative = np.sum(data[:,-1]==0)
    H = 0 
    
    # When only one class of data -> Pure
    if Totalpositive ==0 or Totalnegative ==0:
        return 0
    
    else:
        positive_ratio = Totalpositive/totaldata
        negative_ratio = Totalnegative/totaldata
        H = -positive_ratio * log2(positive_ratio) + (-negative_ratio * log2(negative_ratio))
        return H

In [5]:
def info_gain(data,true_data_index,false_data_index,H):
    """
    Takes data, entropy of that data (H)
    as well as a col number to find info_gain 
    if we split on that col median
    """
    totalDataSize =data.shape[0]
    true_data = data[true_data_index]
    false_data = data[false_data_index]
    
    true_branch_ratio = true_data.shape[0]/totalDataSize
    false_branch_ratio = false_data.shape[0]/totalDataSize  

    if true_branch_ratio == 0 or false_branch_ratio ==0:
        return 0
    
    trueEntropy  = entropy(true_data)
    falseEntropy = entropy(false_data)
    
    I = H - ((true_branch_ratio * trueEntropy) + (false_branch_ratio * falseEntropy))
    return I

In [6]:
def best_split(data,H):
    
    best_info_gain = -1
    best_attr = -1
#     split_median = -1
    trueIndex = None
    falseIndex = None
    # consider all attributes except the last one i.e label *y*
    medianList  = np.median(data[:,:-1],axis =0)

    
    for colIndex, median in enumerate(medianList):
        
        true_data_index = np.where(data[:,colIndex] <= median)
        false_data_index = np.where(data[:,colIndex] > median)

        
        IG = info_gain(data,true_data_index,false_data_index,H)

        if IG > best_info_gain:
            best_info_gain = IG
            best_attr = colIndex
            split_median = median
            trueIndex = true_data_index
            falseIndex = false_data_index

    return best_attr, split_median, trueIndex,falseIndex, best_info_gain

In [7]:
def classCount(data):
    count =dict()
    count[1] = np.sum(data[:,-1]==1)
    count[0] = np.sum(data[:,-1]==0)
    return count

In [8]:
class Decision_nodes:
    """
    Internal nodes where we ask questions 
    to split the node further if possible
    **Also hold the reference to the 2 child branches**
    """
    total_decision_nodes = 0
    depth = 0
    def __init__(self,entropy, best_attr, split_median, depth):
        self.entropy = entropy
        self.best_attr = best_attr
        self.split_median = split_median
        self.depth = depth
        
        Decision_nodes.total_decision_nodes+=1
        if depth > Decision_nodes.depth:
            Decision_nodes.depth = depth

    def assignLabel(self,label):
        self.label = label

In [9]:
class LeafNode:

    total_leaf_nodes = 0
    def __init__(self,data,depth):
        self.prediction = classCount(data)
        self.depth = depth
        if self.prediction[1] > self.prediction[0]:
            self.label = 1
        else:
            self.label = 0
        LeafNode.total_leaf_nodes+=1
        

In [10]:
def Build_tree(data,depth):

    """
    Recursively build the tree
    and stop when there is almost no Inforamtion_gain
    """
    H = entropy(data)
    

    best_attr, split_median, trueIndex, falseIndex, best_info_gain =  best_split(data,H)
 
    # base case to stop recursion
 
    if H ==0 or best_attr == -1 or best_info_gain <= 0:
        return LeafNode(data,depth)
    
    root = Decision_nodes(H,best_attr, split_median, depth)

    true_branch = data[trueIndex]
    false_branch = data[falseIndex]
    
    if true_branch.shape[0] ==0 or false_branch.shape[0]==0:
        return LeafNode(data,depth)
    
    root.leftChild = Build_tree(true_branch,depth+1)
    root.rightChild = Build_tree(false_branch,depth+1)
    return root 

In [11]:
train_data = np.append(train_x,train_y,axis=1)
s = time.time()
root = Build_tree(train_data,0)
print(time.time() -s)

548.6520748138428


In [12]:
print(Decision_nodes.depth)
Decision_nodes.total_decision_nodes

51


9988

In [13]:
LeafNode.total_leaf_nodes


9989

In [18]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, LeafNode):
        return node.label

    col = node.best_attr
    median = node.split_median
    if row[col] <= median:
        return classify(row, node.leftChild)
    else:
        return classify(row, node.rightChild)

In [28]:
prediction = []
for row in train_x:
    prediction.append(classify(row,root))
    

In [29]:
match=0
for i in range(len(prediction)):
    if train_y[i]==prediction[i]:
        match+=1
print((match*100)/len(prediction))

90.43932440158855
